In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
%run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
%run ../../../prepare_data.py
X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
save_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', X_train, y_train, other_features = X_other_features_train)
save_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', X_test, y_test, other_features = X_other_features_test)
save_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', other_features=True)

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
HEREEE!!!
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
HEREEE!!!
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
HEREEE!!!
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [6]:
gen = generate_fake_epoch(X_train, X_other_features_train, y_train, 16)
test = next(gen)
print(np.array(test[0][0]).shape)
for el in test[0][0][7]:
    print(dictionary[el.tolist().index(1)])

(16, 23, 30)
b
o
j
e
v
a
n
j
e
m
a


ValueError: 1 is not in list

In [5]:
print (X_train.shape)
print (X_test.shape)
print (X_validate.shape)
# X_other_features = create_X_features(content)
# print (X_other_features[178200])
decode_position = 38
print (decode_input(X_validate[decode_position], dictionary))
# print (X_other_features_train[0])
print (decode_X_features(feature_dictionary, [X_other_features_train[decode_position]]))
# print (len(X_other_features_train[0]))

(430151, 23, 36)
(52058, 23, 36)
(54222, 23, 36)
najslajši
Agpndi-
Agpndi-


In [6]:
num_examples = len(X_train) # training set size
# nn_input_dim = max_word * len(dictionary) # input layer dimensionality
# nn_output_dim = max_num_vowels * max_num_vowels # output layer dimensionality
nn_output_dim = 11
nn_hdim = 516
batch_size = 16
actual_epoch = 1
num_fake_epoch = 20

# Gradient descent parameters (I picked these by hand)
# epsilon = 1 # learning rate for gradient descent
# reg_lambda = 1 # regularization strength

In [7]:
conv_input_shape=(23, 36)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(43, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(43, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)

In [8]:
model = Model(inputs=[conv_input, othr_input], outputs=x)

# epochs = 5
# lrate = 0.1
# decay = lrate/epochs
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Compile model
# keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [9]:
history = model.fit_generator(generate_fake_epoch(X_train, X_other_features_train, y_train, batch_size), X_train.shape[0]/(batch_size * num_fake_epoch), epochs=actual_epoch*num_fake_epoch, validation_data=([X_test, X_other_features_test], y_test))

# model.fit([X_train, X_other_features_train], y_train, validation_data=([X_validate, X_other_features_validate], y_validate), epochs=1, batch_size=16)

Epoch 1/20
1345/1344 [==============================] - 18s - loss: 0.2051 - actual_accuracy: 0.2616 - val_loss: 0.1565 - val_actual_accuracy: 0.5336
Epoch 2/20
1345/1344 [==============================] - 20s - loss: 0.1457 - actual_accuracy: 0.4864 - val_loss: 0.1292 - val_actual_accuracy: 0.6093
Epoch 3/20
1345/1344 [==============================] - 33s - loss: 0.1217 - actual_accuracy: 0.5961 - val_loss: 0.1140 - val_actual_accuracy: 0.6817
Epoch 4/20
1345/1344 [==============================] - 33s - loss: 0.1117 - actual_accuracy: 0.6520 - val_loss: 0.1066 - val_actual_accuracy: 0.7175
Epoch 5/20
1345/1344 [==============================] - 31s - loss: 0.1056 - actual_accuracy: 0.6833 - val_loss: 0.1017 - val_actual_accuracy: 0.7293
Epoch 6/20
1345/1344 [==============================] - 35s - loss: 0.1005 - actual_accuracy: 0.7048 - val_loss: 0.0964 - val_actual_accuracy: 0.7408
Epoch 7/20
1345/1344 [==============================] - 35s - loss: 0.0952 - actual_accuracy: 0.7251

In [27]:
name = '1_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [4]:
dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
feature_dictionary = create_feature_dictionary()
def generate_input_from_word(word, max_word, dictionary):
    x = np.zeros((max_word, len(dictionary)))
    j = 0
    for c in list(word):
        index = 0
        for d in dictionary:
            if c == d:
                x[j, index] = 1
                break
            index += 1
        j += 1
    return x

# model = load_model()
# prediction = predict(model, generate_input_from_word('hidrija'))
# print decode_position(prediction[0])

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!


In [5]:
# create_syllables(content[375488][0], vowels)
create_syllables('izzvati', vowels)

['iz', 'zva', 'ti']

In [8]:

syllables_dictionary = create_syllables_dictionary(content, vowels)

In [9]:
syllables_dictionary = sorted(syllables_dictionary)
syllables_dictionary

['',
 '-o',
 'a',
 'ab',
 'ah',
 'akt',
 'al',
 'alf',
 'alg',
 'alt',
 'am',
 'an',
 'ant',
 'ar',
 'arh',
 'as',
 'at',
 'av',
 'ba',
 'bab',
 'bah',
 'baj',
 'bajt',
 'bak',
 'bal',
 'bam',
 'ban',
 'banj',
 'bank',
 'bar',
 'bark',
 'barv',
 'bas',
 'bast',
 'bat',
 'bav',
 'baz',
 'bač',
 'baš',
 'baž',
 'bde',
 'bdel',
 'bdet',
 'bdeč',
 'bdi',
 'bdim',
 'bdiš',
 'be',
 'bec',
 'bed',
 'beg',
 'beh',
 'bej',
 'bek',
 'bel',
 'belc',
 'belj',
 'belk',
 'bem',
 'ben',
 'benj',
 'benk',
 'bent',
 'ber',
 'bes',
 'bet',
 'bev',
 'bez',
 'beč',
 'beš',
 'bež',
 'bi',
 'bic',
 'bih',
 'bij',
 'bik',
 'bil',
 'bilj',
 'bilk',
 'bim',
 'bin',
 'bir',
 'bisk',
 'bit',
 'bitk',
 'biv',
 'bič',
 'biš',
 'bišč',
 'bja',
 'bjav',
 'bje',
 'bjej',
 'bjekt',
 'bjel',
 'bjem',
 'bjest',
 'bjet',
 'bješ',
 'bji',
 'bjih',
 'bjim',
 'bjo',
 'bju',
 'bla',
 'blag',
 'blah',
 'blak',
 'blam',
 'blan',
 'blast',
 'blat',
 'ble',
 'bled',
 'blej',
 'blek',
 'blem',
 'blesk',
 'blest',
 'blet',
 'bleč'

In [7]:
%run ../../../prepare_data.py
# generate_X_and_y(dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels, feature_dictionary)